# Secondary Structure Shifted Word2Vec Encoder

This demo creates a dataset of sequence segments derived from a non-redundent set. The dataset contains the seuqence segment, the DSSP Q8 and DSSP Q3 code of the center residue in a sequnece segment, and a 3-gram shifted Word2Vec encoding of the seuqnece segment.

## Imports

In [1]:
from pyspark import SparkConf, SparkContext, SQLContext
from mmtfPyspark.ml import ProteinSequenceEncoder
from mmtfPyspark.mappers import StructureToPolymerChains
from mmtfPyspark.filters import ContainsLProteinChain
from mmtfPyspark.datasets import secondaryStructureSegmentExtractor
from mmtfPyspark.webfilters import Pisces
from mmtfPyspark.io import mmtfReader
import time

## Configure Spark Context

In [2]:
conf = SparkConf() \
        .setMaster("local[*]") \
        .setAppName("secondaryStructureShiftedWord2VecEncodeDemo")
sc = SparkContext(conf = conf)

## Read in, filter and sample Hadoop Sequence Files

In [3]:
path = "../../resources/mmtf_reduced_sample/"

sequenceIdentity = 20
resolution = 2.0
fraction = 0.1
seed = 123

pdb = mmtfReader \
        .read_sequence_file(path, sc) \
        .flatMap(StructureToPolymerChains()) \
        .filter(Pisces(sequenceIdentity, resolution)) \
        .filter(ContainsLProteinChain()) \
        .sample(False, fraction, seed)

## Extract Secondary Structure Segments

In [4]:
segmentLength = 25
data = secondaryStructureSegmentExtractor.get_dataset(pdb, segmentLength).cache()

## Add Word2Vec encoded feature vector

In [6]:
encoder = ProteinSequenceEncoder(data)

windowSize = (segmentLength -1) // 2
vectorSize = 50
# overlapping_ngram_word2vec_encode uses keyword attributes
data = encoder.shifted_3gram_word2vec_encode(windowSize=windowSize, vectorSize=vectorSize).cache()

root
 |-- ngram0: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngram1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngram2: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- structureChainId: string (nullable = false)
 |-- sequence: string (nullable = false)
 |-- labelQ8: string (nullable = false)
 |-- labelQ3: string (nullable = false)
 |-- feature2: vector (nullable = true)
 |-- feature1: vector (nullable = true)
 |-- feature0: vector (nullable = true)
 |-- features: vector (nullable = true)



## Show dataset schema and few rows of data

In [7]:
data.printSchema()
data.show(10, False)

root
 |-- structureChainId: string (nullable = false)
 |-- sequence: string (nullable = false)
 |-- labelQ8: string (nullable = false)
 |-- labelQ3: string (nullable = false)
 |-- ngram0: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngram1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngram2: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- feature0: vector (nullable = true)
 |-- feature1: vector (nullable = true)
 |-- feature2: vector (nullable = true)
 |-- features: vector (nullable = true)

+----------------+-------------------------+-------+-------+----------------------------------------+----------------------------------------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
df = data.toPandas()

df.head(10)

,structureChainId,sequence,labelQ8,labelQ3,ngram0,ngram1,ngram2,feature0,feature1,feature2,features
0,1B8D.K,ACFAKYSYLKNAGEAGDSPEKINKC,T,C,"[ACF, AKY, SYL, KNA, GEA, GDS, PEK, INK]","[CFA, KYS, YLK, NAG, EAG, DSP, EKI, NKC]","[FAK, YSY, LKN, AGE, AGD, SPE, KIN, KC]","[-2.5167602449655533, 0.41110057570040226, -0....","[-0.9313944121822715, 0.8716795854270458, -2.8...","[-0.3790398994460702, 2.511866755783558, -3.56...","[-1.2757315188646317, 1.264882305637002, -2.46..."
1,1AOL.A,CNTAWNRLKLDQVTHKSSEGFYVCP,H,H,"[CNT, AWN, RLK, LDQ, VTH, KSS, EGF, YVC]","[NTA, WNR, LKL, DQV, THK, SSE, GFY, VCP]","[TAW, NRL, KLD, QVT, HKS, SEG, FYV, CP]","[-0.8749945722520351, -2.3850620687007904, -0....","[-1.5751862302422523, 2.3965483605861664, -1.9...","[-0.9588506408035755, -0.6263859607279301, 3.7...","[-1.136343814432621, -0.20496655628085136, 0.2..."
2,1FO8.A,DPSLWCVSAWNDNGKEQMVDSSKPE,T,C,"[DPS, LWC, VSA, WND, NGK, EQM, VDS, SKP]","[PSL, WCV, SAW, NDN, GKE, QMV, DSS, KPE]","[SLW, CVS, AWN, DNG, KEQ, MVD, SSK, PE]","[0.11070175841450691, -1.486862190067768, -1.5...","[1.1518815904855728, -0.13194116204977036, 3.5...","[-0.0534269493073225, -1.546366237103939, -0.8...","[0.40305213319758576, -1.0550565297404926, 0.3..."
3,1FSG.C,GCCYDFNEMFRDFDHVAVLSDAARK,C,C,"[GCC, YDF, NEM, FRD, FDH, VAV, LSD, AAR]","[CCY, DFN, EMF, RDF, DHV, AVL, SDA, ARK]","[CYD, FNE, MFR, DFD, HVA, VLS, DAA, RK]","[2.2771085798740387, -1.5183069882914424, -0.5...","[2.024966076016426, 0.7575673609972, -1.057440...","[1.3174011707305908, -0.8285085475072265, -0.4...","[1.8731586088736851, -0.5297493916004896, -0.6..."
4,1FSG.C,IDKILLPGGLVKDRVEKLAYDIHRT,H,H,"[IDK, ILL, PGG, LVK, DRV, EKL, AYD, IHR]","[DKI, LLP, GGL, VKD, RVE, KLA, YDI, HRT]","[KIL, LPG, GLV, KDR, VEK, LAY, DIH, RT]","[0.0011718850582838058, -0.8450929443351924, 2...","[0.6603811718523502, -1.688733596354723, 0.210...","[-0.14117270801216364, 0.501228928565979, -0.4...","[0.17346011629949012, -0.6775325373746455, 0.6..."
5,1C1K.A,KAYRKILNIDSQKAKNVFIETVKSC,H,H,"[KAY, RKI, LNI, DSQ, KAK, NVF, IET, VKS]","[AYR, KIL, NID, SQK, AKN, VFI, ETV, KSC]","[YRK, ILN, IDS, QKA, KNV, FIE, TVK, SC]","[0.8315972853451967, 0.043749348260462284, 0.9...","[-0.14495163690298796, -1.5973218269646168, 1....","[-0.09335422236472368, -2.975793592631817, 0.4...","[0.19776380869249502, -1.5097886904453237, 0.8..."
6,1A9X.F,LSSYLKRHNIVAIADIDTRKLTRLL,E,E,"[LSS, YLK, RHN, IVA, IAD, IDT, RKL, TRL]","[SSY, LKR, HNI, VAI, ADI, DTR, KLT, RLL]","[SYL, KRH, NIV, AIA, DID, TRK, LTR, LL]","[2.108544853515923, 2.1560158729553223, -3.113...","[-1.3656953491736203, 2.262380998581648, -0.14...","[-0.920694176107645, -0.6884733289480209, 0.31...","[-0.059281557255114116, 1.2433078475296497, -0..."
7,1B8D.K,SAYIAAFTFTRDRLCVPRDMSSQAG,H,H,"[SAY, IAA, FTF, TRD, RLC, VPR, DMS, SQA]","[AYI, AAF, TFT, RDR, LCV, PRD, MSS, QAG]","[YIA, AFT, FTR, DRL, CVP, RDM, SSQ, AG]","[-0.2355625682976097, 0.7051106672734022, -1.9...","[1.7952079474925995, 0.06272168457508087, -2.5...","[0.07700180681422353, 1.658301018178463, -1.82...","[0.5455490620030711, 0.8087111233423153, -2.11..."
8,1A9X.F,TTAEAYSWTQGSWTLTGGLPQAKKE,C,C,"[TTA, EAY, SWT, QGS, WTL, TGG, LPQ, AKK]","[TAE, AYS, WTQ, GSW, TLT, GGL, PQA, KKE]","[AEA, YSW, TQG, SWT, LTG, GLP, QAK, KE]","[-0.5131183154881, -1.3824960142374039, 0.2537...","[-0.7859331574290991, -2.3178150579333305, 2.6...","[-0.1625298084691167, 0.3564658295363188, -0.8...","[-0.4871937604621053, -1.1146150808781385, 0.6..."
9,1FPO.B,ADTCRKLRFLDKLRSSAEQLEEKLL,H,H,"[ADT, CRK, LRF, LDK, LRS, SAE, QLE, EKL]","[DTC, RKL, RFL, DKL, RSS, AEQ, LEE, KLL]","[TCR, KLR, FLD, KLR, SSA, EQL, EEK, LL]","[0.5554861556738615, -1.1574508603662252, 1.13...","[0.33273782185278833, -2.0636640461161733, 1.2...","[0.20934702455997467, -0.699911005795002, -0.9...","[0.3658570006955415, -1.3070086374258, 0.47036..."


## Terminate Spark Context

In [9]:
sc.stop()